In [192]:
import polars as pl
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

c:\Users\Lauritz\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 4>                                                                                │
│                                                                                                  │
│   1 import polars as pl                                                                          │
│   2 from transformers import BertTokenizer, BertModel                                            │
│   3 tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')                               │
│ ❱ 4 model = BertModel.from_pretrained("bert-base-uncased")                                       │
│   5                                                                                              │
│                                                                                                  │
│ c:\Users\Lauritz\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\modeling │
│ _utils.py:2301 in from_pretrained                                                                │
│                                                                                                  │
│   2298 │   │   if from_pt:                                                                       │
│   2299 │   │   │   if not is_sharded and state_dict is None:                                     │
│   2300 │   │   │   │   # Time to load the checkpoint                                             │
│ ❱ 2301 │   │   │   │   state_dict = load_state_dict(resolved_archive_file)                       │
│   2302 │   │   │                                                                                 │
│   2303 │   │   │   # set dtype to instantiate the model under:                                   │
│   2304 │   │   │   # 1. If torch_dtype is not None, we use that dtype                            │
│                                                                                                  │
│ c:\Users\Lauritz\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\modeling │
│ _utils.py:402 in load_state_dict                                                                 │
│                                                                                                  │
│    399 │   """                                                                                   │
│    400 │   if checkpoint_file.endswith(".safetensors") and is_safetensors_available():           │
│    401 │   │   # Check format of the archive                                                     │
│ ❱  402 │   │   with safe_open(checkpoint_file, framework="pt") as f:                             │
│    403 │   │   │   metadata = f.metadata()                                                       │
│    404 │   │   if metadata.get("format") not in ["pt", "tf", "flax"]:                            │
│    405 │   │   │   raise OSError(                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
OSError: Sidefilen er for lille til, at denne handling kan gennemføres. (os error 1455)

In [136]:
def get_mean_bert_embedding(texts):
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    output = model(**encoded_input)
    sentence_embedding = output.last_hidden_state.mean(dim=1)
    return sentence_embedding.detach().numpy()

In [30]:
df = pl.scan_csv('../../data/clean/rotten_tomatoes_movie_details_clean.csv')

In [164]:
df.collect().shape

(17661, 4)

In [45]:
((17661/5)*1.9)/60

111.853

In [71]:
(30.3 * (17661/100))/60

89.18805

In [152]:
import polars as pl
import numpy as np

def embed_batch(df: pl.DataFrame) -> pl.DataFrame:
    texts = df["description"].to_list()
    emb = get_mean_bert_embedding(texts)  # numpy array, shape: (batch, 768)

    # Convert numpy rows → Python lists (Polars can store these)
    emb_as_lists = [row.tolist() for row in emb]

    return pl.DataFrame(
        {"embedding": emb_as_lists},
        schema={"embedding": pl.Array(768, pl.Float64)}
    )


In [186]:
def embed_batch(df: pl.DataFrame) -> pl.DataFrame:
    # Ensure pure Python strings
    texts = [str(x) for x in df["description"].to_list()]
    
    emb = get_mean_bert_embedding(texts)  # shape (batch, 768)
    return pl.DataFrame({"embedding": [row.tolist() for row in emb]})


In [187]:
out = df.map_batches(embed_batch)


In [191]:
out.head(1).collect()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>                                                                                │
│                                                                                                  │
│ ❱ 1 out.head(1).collect()                                                                        │
│   2                                                                                              │
│                                                                                                  │
│ c:\Users\Lauritz\AppData\Local\Programs\Python\Python310\lib\site-packages\polars\_utils\depreca │
│ tion.py:97 in wrapper                                                                            │
│                                                                                                  │
│    94 │   │   │   │                                                                              │
│    95 │   │   │   │   del kwargs["streaming"]                                                    │
│    96 │   │   │                                                                                  │
│ ❱  97 │   │   │   return function(*args, **kwargs)                                               │
│    98 │   │                                                                                      │
│    99 │   │   wrapper.__signature__ = inspect.signature(function)  # type: ignore[attr-defined   │
│   100 │   │   return wrapper                                                                     │
│                                                                                                  │
│ c:\Users\Lauritz\AppData\Local\Programs\Python\Python310\lib\site-packages\polars\lazyframe\opt_ │
│ flags.py:328 in wrapper                                                                          │
│                                                                                                  │
│   325 │   │   │   │   │   optflags = cb(optflags, kwargs.pop(key))  # type: ignore[no-untyped-   │
│   326 │   │   │                                                                                  │
│   327 │   │   │   kwargs["optimizations"] = optflags                                             │
│ ❱ 328 │   │   │   return function(*args, **kwargs)                                               │
│   329 │   │                                                                                      │
│   330 │   │   wrapper.__signature__ = inspect.signature(function)  # type: ignore[attr-defined   │
│   331 │   │   return wrapper                                                                     │
│                                                                                                  │
│ c:\Users\Lauritz\AppData\Local\Programs\Python\Python310\lib\site-packages\polars\lazyframe\fram │
│ e.py:2422 in collect                                                                             │
│                                                                                                  │
│   2419 │   │                                                                                     │
│   2420 │   │   # Only for testing purposes                                                       │
│   2421 │   │   callback = _kwargs.get("post_opt_callback", callback)                             │
│ ❱ 2422 │   │   return wrap_df(ldf.collect(engine, callback))                                     │
│   2423 │                                                                                         │
│   2424 │   @overload                                                                             │
│   2425 │   def collect_async(                                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ComputeError: The output schema of 'LazyFrame.map' is incorre

In [189]:
df_e = out.collect()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>                                                                                │
│                                                                                                  │
│ ❱ 1 df_e = out.collect()                                                                         │
│   2                                                                                              │
│                                                                                                  │
│ c:\Users\Lauritz\AppData\Local\Programs\Python\Python310\lib\site-packages\polars\_utils\depreca │
│ tion.py:97 in wrapper                                                                            │
│                                                                                                  │
│    94 │   │   │   │                                                                              │
│    95 │   │   │   │   del kwargs["streaming"]                                                    │
│    96 │   │   │                                                                                  │
│ ❱  97 │   │   │   return function(*args, **kwargs)                                               │
│    98 │   │                                                                                      │
│    99 │   │   wrapper.__signature__ = inspect.signature(function)  # type: ignore[attr-defined   │
│   100 │   │   return wrapper                                                                     │
│                                                                                                  │
│ c:\Users\Lauritz\AppData\Local\Programs\Python\Python310\lib\site-packages\polars\lazyframe\opt_ │
│ flags.py:328 in wrapper                                                                          │
│                                                                                                  │
│   325 │   │   │   │   │   optflags = cb(optflags, kwargs.pop(key))  # type: ignore[no-untyped-   │
│   326 │   │   │                                                                                  │
│   327 │   │   │   kwargs["optimizations"] = optflags                                             │
│ ❱ 328 │   │   │   return function(*args, **kwargs)                                               │
│   329 │   │                                                                                      │
│   330 │   │   wrapper.__signature__ = inspect.signature(function)  # type: ignore[attr-defined   │
│   331 │   │   return wrapper                                                                     │
│                                                                                                  │
│ c:\Users\Lauritz\AppData\Local\Programs\Python\Python310\lib\site-packages\polars\lazyframe\fram │
│ e.py:2422 in collect                                                                             │
│                                                                                                  │
│   2419 │   │                                                                                     │
│   2420 │   │   # Only for testing purposes                                                       │
│   2421 │   │   callback = _kwargs.get("post_opt_callback", callback)                             │
│ ❱ 2422 │   │   return wrap_df(ldf.collect(engine, callback))                                     │
│   2423 │                                                                                         │
│   2424 │   @overload                                                                             │
│   2425 │   def collect_async(                                                                    │
│                                                                                                  │
│ in embed_batch                                             

In [175]:
out = df.select(pl.col("description").map_batches(embed_batch, return_dtype=pl.List(pl.Float64))).head(10)

In [177]:
print("type(out):", type(out))
print("type(df):", type(df))
import polars as pl
print("polars version:", pl.__version__)

type(out): <class 'polars.lazyframe.frame.LazyFrame'>
type(df): <class 'polars.lazyframe.frame.LazyFrame'>
polars version: 1.35.2


In [176]:
df_e = out.collect()  

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>                                                                                │
│                                                                                                  │
│ ❱ 1 df_e = out.collect()                                                                         │
│   2                                                                                              │
│                                                                                                  │
│ c:\Users\Lauritz\AppData\Local\Programs\Python\Python310\lib\site-packages\polars\_utils\depreca │
│ tion.py:97 in wrapper                                                                            │
│                                                                                                  │
│    94 │   │   │   │                                                                              │
│    95 │   │   │   │   del kwargs["streaming"]                                                    │
│    96 │   │   │                                                                                  │
│ ❱  97 │   │   │   return function(*args, **kwargs)                                               │
│    98 │   │                                                                                      │
│    99 │   │   wrapper.__signature__ = inspect.signature(function)  # type: ignore[attr-defined   │
│   100 │   │   return wrapper                                                                     │
│                                                                                                  │
│ c:\Users\Lauritz\AppData\Local\Programs\Python\Python310\lib\site-packages\polars\lazyframe\opt_ │
│ flags.py:328 in wrapper                                                                          │
│                                                                                                  │
│   325 │   │   │   │   │   optflags = cb(optflags, kwargs.pop(key))  # type: ignore[no-untyped-   │
│   326 │   │   │                                                                                  │
│   327 │   │   │   kwargs["optimizations"] = optflags                                             │
│ ❱ 328 │   │   │   return function(*args, **kwargs)                                               │
│   329 │   │                                                                                      │
│   330 │   │   wrapper.__signature__ = inspect.signature(function)  # type: ignore[attr-defined   │
│   331 │   │   return wrapper                                                                     │
│                                                                                                  │
│ c:\Users\Lauritz\AppData\Local\Programs\Python\Python310\lib\site-packages\polars\lazyframe\fram │
│ e.py:2422 in collect                                                                             │
│                                                                                                  │
│   2419 │   │                                                                                     │
│   2420 │   │   # Only for testing purposes                                                       │
│   2421 │   │   callback = _kwargs.get("post_opt_callback", callback)                             │
│ ❱ 2422 │   │   return wrap_df(ldf.collect(engine, callback))                                     │
│   2423 │                                                                                         │
│   2424 │   @overload                                                                             │
│   2425 │   def collect_async(                                                                    │
│                                                                                                  │
│ c:\Users\Lauritz\AppData\Local\Programs\Python\Python310\li

In [139]:
out = df.select(pl.col('description').map_batches(get_mean_bert_embedding)).head(100).collect()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>                                                                                │
│                                                                                                  │
│ ❱ 1 out = df.select(pl.col('description').map_batches(get_mean_bert_embedding)).head(100).co     │
│   2                                                                                              │
│                                                                                                  │
│ c:\Users\Lauritz\AppData\Local\Programs\Python\Python310\lib\site-packages\polars\_utils\depreca │
│ tion.py:97 in wrapper                                                                            │
│                                                                                                  │
│    94 │   │   │   │                                                                              │
│    95 │   │   │   │   del kwargs["streaming"]                                                    │
│    96 │   │   │                                                                                  │
│ ❱  97 │   │   │   return function(*args, **kwargs)                                               │
│    98 │   │                                                                                      │
│    99 │   │   wrapper.__signature__ = inspect.signature(function)  # type: ignore[attr-defined   │
│   100 │   │   return wrapper                                                                     │
│                                                                                                  │
│ c:\Users\Lauritz\AppData\Local\Programs\Python\Python310\lib\site-packages\polars\lazyframe\opt_ │
│ flags.py:328 in wrapper                                                                          │
│                                                                                                  │
│   325 │   │   │   │   │   optflags = cb(optflags, kwargs.pop(key))  # type: ignore[no-untyped-   │
│   326 │   │   │                                                                                  │
│   327 │   │   │   kwargs["optimizations"] = optflags                                             │
│ ❱ 328 │   │   │   return function(*args, **kwargs)                                               │
│   329 │   │                                                                                      │
│   330 │   │   wrapper.__signature__ = inspect.signature(function)  # type: ignore[attr-defined   │
│   331 │   │   return wrapper                                                                     │
│                                                                                                  │
│ c:\Users\Lauritz\AppData\Local\Programs\Python\Python310\lib\site-packages\polars\lazyframe\fram │
│ e.py:2422 in collect                                                                             │
│                                                                                                  │
│   2419 │   │                                                                                     │
│   2420 │   │   # Only for testing purposes                                                       │
│   2421 │   │   callback = _kwargs.get("post_opt_callback", callback)                             │
│ ❱ 2422 │   │   return wrap_df(ldf.collect(engine, callback))                                     │
│   2423 │                                                                                         │
│   2424 │   @overload                                                                             │
│   2425 │   def collect_async(                                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
InvalidOperationError: UDF called without return type, but wa

In [69]:
out.estimated_size("mb") * (17661/100)

50.843319025039676